<a href="https://colab.research.google.com/github/sngo/llms-practice/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#prepare for import
!pip install gradio==4.44.1

In [2]:
import os
from google.colab import userdata
from typing import List
from openai import OpenAI
import gradio as gr

In [3]:
openai_api_key = userdata.get("OPENAI_API_KEY")

In [4]:
openai = OpenAI(api_key=openai_api_key)
openai_model = 'gpt-4.1-nano'

We will write a function chat(message, history) where:

message is the prompt to use

history is the past conversation, in OpenAI format




In [5]:
system_message = "You are a helpful assistant."

In [6]:
def chat(message, history):
    #format of this message will only work on OPENAI.  Need to test with all of other models
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=openai_model, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True, debug=True)

In [8]:
system_message = "You are a helpful assistant in a clothing store. Gently encourage customers to explore items that are on sale. \
Here are the current discounts: Hats are 60% off, sunglasses are 70% off, jackets are 50% off, and scarves are 40% off. \
Please note that some items, like shoes and belts, are not currently on sale. \
If a customer expresses interest in something like a hat, you could say, 'Great choice — we have a fantastic selection of hats, and many are 60% off as part of our current promotion!' \
If the customer is uncertain about what to get, you can suggest they check out the discounted items, especially hats and sunglasses."

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True, debug=True)

In [10]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [11]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."

    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=openai_model, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()